In [212]:
import os 
import sys
import glob
import nibabel as nib
import numpy as np
import pandas as pd
from scipy import ndimage
from matplotlib.path import Path
from subprocess import call
from dipy.io.image import load_nifti, save_nifti
from dipy.align.reslice import reslice
from shutil import copyfile
from subprocess import Popen, PIPE
import csv 
from shutil import copy, move

This script batch process T1mapping. It includes registering c23 PSIR to 1 slice of T1scout, then apply such transformation to rois which were converted to nii ahead. And get metrics for T1scout and T1mapping images on these rois. 

## make a manifest spreadsheet to specify the inputs 

In [223]:
latest_manifest_path = "/data/henry10/nico/T1mapping_MP2RAGE/script_t1mapping/manifest_psir.xlsx"
sheet_name = "PSIR"
psir_df = pd.read_excel(latest_manifest_path, sheet_name=sheet_name,
                                    dtype={'file_path': str, 'psir': str, 'T1scout': str})
batch_df = psir_df[psir_df.ready == True]
print(f'{batch_df.shape[0]} records to process')

1 records to process


## define functions: roi to nifti convertion, sct_register_multimudal for c23 psir and T1scout at same timepoint, applywarp to rois, get and write median and mean metrics

In [224]:
def jim_roi_to_nifti(jim_roi, ref, name):
    print ('collating points from %s roi file'%jim_roi)
    rois = 0 
    roilist = []
    roifile = open(jim_roi, 'r')
    lines = roifile.readlines()
    roifile.close
    for line in lines:
        line = line.strip()
        if line.startswith('Slice'):
            z = line.split('=')[-1]
            rois +=1
        if line.startswith('X'):
            parts = line.split('=')
            x = parts[1].split(';')[0]
            y = parts[-1]
            roilist += [[x, -float(y), z, rois]]
    pts = np.array(roilist, dtype='float64')
    pts = pts[pts[:,3]==1]
    #return pts 

    print ('transforming points from jim to reference space')
    img = nib.load(ref)
    affine = img.get_affine()
    hdr = img.get_header()
    voxdims = hdr.get_zooms()
    fov = hdr.get_data_shape()
    pts[:,0] = pts[:,0]*(1.0/voxdims[0])+fov[0]/2.0
    pts[:,1] = pts[:,1]*(1.0/voxdims[1])+fov[1]/2.0
    #return pts, affine, fov, voxdims

    print ('filling roi')
    roi_sort = {}
    for pt in pts:
        roi = pt[-1]
        if roi not in roi_sort.keys():
            roi_sort[roi] = {}
        if 'slice' not in roi_sort[roi].keys():
            roi_sort[roi]['slice'] = pt[2]
        if 'pts' in roi_sort[roi].keys():
            roi_sort[roi]['pts'].append(pt[:2])
        else:
            roi_sort[roi]['pts'] = [pt[:2]]
    roi = np.zeros(fov)
    for k, z in enumerate(roi_sort.keys()):
        path = Path(roi_sort[z]['pts'])
        for x in np.arange(fov[0]):
            for y in np.arange(fov[1]):
                if roi_sort[z]['slice']-2 != 1:
                    roi[x,y] = path.contains_point([x+0.5, y+0.5])
        print ('completed iterating through roi %i/%i' % (k+1,len(roi_sort.keys())))
    #return roi

    print ('saving roi image')
    new_roi = nib.Nifti1Image(roi, affine)
    New_ROI = nib.save(new_roi, os.path.join(file_path, '{}.nii.gz').format(name))
    Nif_ROI = os.path.join(file_path, '{}.nii.gz').format(name)
    return Nif_ROI

In [225]:
def reorder_T1():
    img = nib.load(T1scout)
    data = img.get_fdata()
    new_data = data.reshape(data.shape[0], data.shape[1], 1, data.shape[2])
    new_img = nib.Nifti1Image(new_data, img.affine)
    T1scout_t = T1scout[:-7]+'t.nii.gz'
    nib.save(new_img, T1scout_t)
    return T1scout_t 

In [226]:
def sct_register_multimodal():
    T1 = os.path.join(file_path, '{}_vol5.nii.gz'.format(mseID))
    call(['fslroi', T1scout, T1, '0', '-1', '0', '-1', '5', '1'])
    T1stack = os.path.join(file_path, '{}_vol5-6.nii.gz'.format(mseID))
    call(['fslmerge', '-z', T1stack, T1, T1, T1, T1, T1, T1])

    psirstack = os.path.join(file_path, '{}_PSIR.nii.gz'.format(mseID))
    call(['fslmerge', '-z', psirstack, psir, psir, psir, psir, psir])

    call(['sct_get_centerline', '-i', psirstack, '-c', 't1', '-method', 'optic'])
    call(['sct_get_centerline', '-i', T1stack, '-c', 't2', '-method', 'optic'])
    qc = os.path.join(file_path, 'qc')
    call(['sct_deepseg_sc', '-i', psirstack, '-c', 't1', '-ofolder', file_path, '-qc', qc])
    call(['sct_deepseg_sc', '-i', T1stack, '-c', 't2', '-ofolder', file_path, '-qc', qc])
    call(['rm', '-r', qc])
 
    T1_centerline = T1stack[:-7]+'_centerline.nii.gz'
    T1_seg = T1stack[:-7]+'_seg.nii.gz'
    psir_centerline = psirstack[:-7]+'_centerline.nii.gz'
    psir_seg = psirstack[:-7]+'_seg.nii.gz'       
    #register PSIR to T1 spacs
    call(['sct_register_multimodal', '-i', psirstack, '-iseg', psir_seg, '-ilabel', psir_centerline, '-d', T1stack, '-dseg', T1_seg, '-dlabel', T1_centerline, '-ofolder', file_path, '-param', 'step=1,type=seg,algo=centermass:step=2,type=seg,algo=bsplinesyn,slicewise=1,iter=3'])
    warp = glob.glob(os.path.join(file_path, 'warp_{}_PSIR*').format(mseID))[0]
    print(warp)
    psir_reg = psirstack[:-7]+'_reg.nii.gz'
    T1_reg = T1stack[:-7]+'_reg.nii.gz'
    call(['fslroi', psir_reg, psir_reg, '0', '-1', '0', '-1', '0', '1'])
    #delete tmp files
    T1_centerline_csv = T1stack[:-7]+'_centerline.csv'
    psir_centerline_csv = psirstack[:-7]+'_centerline.csv'
    call(['rm', T1, psirstack, T1_reg, T1_centerline, T1_seg, psir_centerline, psir_seg, T1_centerline_csv, psir_centerline_csv])
    return warp, T1stack

In [227]:
def warp_masks(mask, name):
    mask_5 = mask[:-7]+'_5.nii.gz'
    call(['fslmerge', '-z', mask_5, mask, mask, mask, mask, mask])
    mask_reg = os.path.join(file_path, '{}_{}_reg.nii.gz'.format(mseID, name))
    call(['sct_apply_transfo', '-i', mask_5, '-d', T1stack, '-w', warp, '-o', mask_reg, '-x', 'nn']) 
    call(['fslroi', mask_reg, mask_reg, '0', '-1', '0', '-1', '0', '1'])
    call(['rm', mask_5])
    return mask_reg

In [228]:
def get_median_values(image, mask):
    cmd = ["fslstats", image, "-k", mask, "-P", "50"]
    proc = Popen(cmd, stdout=PIPE)
    P_50 = [l.decode("utf-8").split() for l in proc.stdout.readlines()]
    P_50 = str(P_50)
    P_50 = P_50.replace("'","").replace("[[","").replace("]]","")
    return P_50
def get_mean_values(image, mask):
    cmd = ["fslstats", image, "-k", mask, "-M"]
    proc = Popen(cmd, stdout=PIPE)
    MEAN = [l.decode("utf-8").split() for l in proc.stdout.readlines()]
    MEAN = str(MEAN)
    MEAN = MEAN.replace("'","").replace("[[","").replace("]]","")
    return MEAN
def write_to_csv(image):
    CSV = os.path.join(file_path, '{}_metrics.csv'.format(mseID))
    image_WM_median = get_median_values(image, WM_reg)
    image_GM_median = get_median_values(image, GM_reg)
    image_CSF_median = get_median_values(image, CSF_reg)
    image_Lesion_median = get_median_values(image, LESION_reg)
    image_WM_mean = get_mean_values(image, WM_reg)
    image_GM_mean = get_mean_values(image, GM_reg)
    image_CSF_mean = get_mean_values(image, CSF_reg)
    image_Lesion_mean = get_mean_values(image, LESION_reg)
    new_row = [image, image_WM_median, image_GM_median, image_CSF_median, image_Lesion_median, image_WM_mean, image_GM_mean, image_CSF_mean, image_Lesion_mean]
    header = ['image_file', 'WM_median', 'GM_median', 'CSF_median', 'Lesion_medain', 'WM_mean', 'GM_mean', 'CSF_mean', 'Lesion_mean']
    file_exists = os.path.isfile(CSV)
    file_empty = os.stat(CSV).st_size == 0 if file_exists else True
    with open(CSV, 'a') as f:
        writer = csv.writer(f)
        if file_empty:
            writer.writerow(header)
        writer.writerow(new_row)
        print(new_row)

In [229]:
def get_median_values_t(image, mask):
    cmd = ["fslstats", '-t', image, "-k", mask, "-P", "50"]
    proc = Popen(cmd, stdout=PIPE)
    P_50 = [l.decode("utf-8").split() for l in proc.stdout.readlines()]
    return P_50
def get_mean_values_t(image, mask):
    cmd = ["fslstats", '-t', image, "-k", mask, "-M"]
    proc = Popen(cmd, stdout=PIPE)
    MEAN = [l.decode("utf-8").split() for l in proc.stdout.readlines()]
    return MEAN
def write_to_csv_t(image):
    CSV = os.path.join(file_path, '{}_metrics.csv'.format(mseID))
    image_WM_median = get_median_values_t(image, WM_reg)
    image_GM_median = get_median_values_t(image, GM_reg)
    image_CSF_median = get_median_values_t(image, CSF_reg)
    image_Lesion_median = get_median_values_t(image, LESION_reg)
    image_WM_mean = get_mean_values_t(image, WM_reg)
    image_GM_mean = get_mean_values_t(image, GM_reg)
    image_CSF_mean = get_mean_values_t(image, CSF_reg)
    image_Lesion_mean = get_mean_values_t(image, LESION_reg)
    for i in range(23):
        new_row = [image[:-7]+'_{}.nii.gz'.format(i), image_WM_median[i][0], image_GM_median[i][0], image_CSF_median[i][0], image_Lesion_median[i][0], image_WM_mean[i][0], image_GM_mean[i][0], image_CSF_mean[i][0], image_Lesion_mean[i][0]]
        with open(CSV, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(new_row)
            print(new_row)

## iterate to batch processing 

In [230]:
from tqdm import tqdm
for index, row in tqdm(batch_df.iterrows(), total=batch_df.shape[0]):
    file_path = batch_df.iloc[index]["file_path"]
    psir_name = batch_df.iloc[index]["psir"]
    T1scout_name = batch_df.iloc[index]["T1scout"]
    mseID = batch_df.iloc[index]["mseID"]
    T1mapping_name = batch_df.iloc[index]["T1mapping"]
    T1mappingA_name = batch_df.iloc[index]["T1mappingA"]
    T1mappingB_name = batch_df.iloc[index]["T1mappingB"]
    T1mappingC_name = batch_df.iloc[index]["T1mappingC"]
    print(file_path)
    
    # input images 
    psir = glob.glob(os.path.join(file_path, psir_name))[0]
    T1scout = glob.glob(os.path.join(file_path, T1scout_name))[0]
    WMroi = glob.glob(os.path.join(file_path, 'WM.roi'))[0]
    GMroi = glob.glob(os.path.join(file_path, 'GM.roi'))[0]
    CSFroi = glob.glob(os.path.join(file_path, 'CSF.roi'))[0]
    LESIONroi = glob.glob(os.path.join(file_path, 'lesion.roi'))[0]
    
    # STEP 1: convert roi to nifti 
    WM = jim_roi_to_nifti(WMroi, psir, 'WM')
    GM = jim_roi_to_nifti(GMroi, psir, 'GM')
    CSF = jim_roi_to_nifti(CSFroi, psir, 'CSF')
    LESION = jim_roi_to_nifti(LESIONroi, psir, 'lesion')
    
    # STEP 2: register c23 psir to T1scout slice 22
    T1scout_t = reorder_T1()
    warp, T1stack = sct_register_multimodal()
    
    # STEP 3: apply warp_c23_PSIR_t1scout to rois
    WM_reg = warp_masks(WM, 'WM')
    GM_reg = warp_masks(GM, 'GM')
    CSF_reg = warp_masks(CSF, 'CSF')
    LESION_reg = warp_masks(LESION, 'lesion')
    print(LESION_reg)
    call(['rm', T1stack])
    
    # Step 4: get metrics
    T1mapping = glob.glob(os.path.join(file_path, T1mapping_name))[0]
    T1mappingA = glob.glob(os.path.join(file_path, T1mappingA_name))[0]
    T1mappingB = glob.glob(os.path.join(file_path, T1mappingB_name))[0]
    T1mappingC = glob.glob(os.path.join(file_path, T1mappingC_name))[0]
    print(T1mappingC)
    for i in [T1mapping, T1mappingA, T1mappingB, T1mappingC]:
        write_to_csv(i)
    write_to_csv_t(T1scout_t)

  0%|          | 0/1 [00:00<?, ?it/s]/data/henry4/pbr_env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/data/henry4/pbr_env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135
collating points from /data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/WM.roi roi file
transforming points from jim to reference space
filling roi
completed iterating through roi 1/1
saving roi image
collating points from /data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/GM.roi roi file
transforming points from jim to reference space
filling roi
completed iterating through roi 1/1
saving roi image
collating points from /data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/CSF.roi roi file
transforming points from jim to reference space
filling roi
completed iterating through roi 1/1
saving roi image
collating points from /data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/lesion.roi roi file
transforming points from jim to reference space
filling roi
completed iterating through roi 1/1
saving roi image
/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/warp_mse6135_PSIR2mse6135_vol5-6.nii.gz
/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/m

100%|██████████| 1/1 [02:33<00:00, 153.37s/it]

['/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/ms6135-mse6135-031_pre_FA15_axial_C23_t1_mapping_2000_gating_92709_31_t_0.nii.gz', '105.000000', '124.000000', '90.000000', '121.000000', '103.400000', '121.000000', '88.066667', '120.000000']
['/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/ms6135-mse6135-031_pre_FA15_axial_C23_t1_mapping_2000_gating_92709_31_t_1.nii.gz', '85.000000', '102.000000', '85.000000', '98.000000', '86.000000', '100.000000', '83.933333', '98.666667']
['/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/ms6135-mse6135-031_pre_FA15_axial_C23_t1_mapping_2000_gating_92709_31_t_2.nii.gz', '69.000000', '88.000000', '80.000000', '90.000000', '69.000000', '87.000000', '78.200000', '90.333333']
['/data/henry10/nico/T1mapping_MP2RAGE/older/ms6135/ms6135-mse6135-031_pre_FA15_axial_C23_t1_mapping_2000_gating_92709_31_t_3.nii.gz', '53.000000', '78.000000', '72.000000', '73.000000', '53.800000', '75.250000', '72.333333', '73.666667']
['/data/henry10/nico/T1mapping_MP2RA